In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
        
from statsmodels.tsa.arima_model import ARIMA               
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [2]:
#series=pd.read_csv("080289/chap07/data/sales.csv")
#series.head()

In [3]:
#index_col=0: 첫 번째 열을 데이터프레임의 인덱스로 사용합니다. 이는 일반적으로 날짜나 시간 정보가 포함된 열입니다.
#parse_dates=[0]: 첫 번째 열을 날짜로 파싱합니다. 이를 통해 pandas는 이 열을 datetime 객체로 변환하여 시계열 데이터로 처리합니다.
series=pd.read_csv("080289/chap07/data/sales.csv", index_col=0, date_format="%Y-%m", parse_dates=[0])
series.head()

,Sales
Month,
1-01,266.0
1-02,145.9
1-03,183.1
1-04,119.3
1-05,180.3


In [10]:
series.index

Index(['1-01', '1-02', '1-03', '1-04', '1-05', '1-06', '1-07', '1-08', '1-09',
       '1-10', '1-11', '1-12', '2-01', '2-02', '2-03', '2-04', '2-05', '2-06',
       '2-07', '2-08', '2-09', '2-10', '2-11', '2-12', '3-01', '3-02', '3-03',
       '3-04', '3-05', '3-06', '3-07', '3-08', '3-09', '3-10', '3-11', '3-12'],
      dtype='object', name='Month')

In [4]:

X=series.values                               # 인덱스 Month 제외한 값 : Sales
X=np.nan_to_num(X)                           #NaN 값을 0으로 변환합니다.
size = int(len(X) * 0.66)                    #여기서 66%는 훈련 세트에, 나머지 34%는 테스트 세트에 할당됩니다.
train, test = X[0:size], X[size:len(X)]

# 히스토리 및 예측 초기화
history=[x for x in train]
predictions=[]

In [5]:
size, len(X)

(23, 36)

훈련 세트 (train): 데이터의 처음 23개 샘플입니다. 즉, X[0:size]는 X[0:23]으로, 시계열의 처음부터 23번째 샘플까지 포함합니다.  <br />
테스트 세트 (test): 데이터의 나머지 부분입니다. 즉, X[size:len(X)]는 X[23:36]으로, 24번째 샘플부터 마지막 36번째 샘플까지 포함합니다.

In [6]:
for t in range(len(test)):
    model=SARIMAX(history, order=(5,1,0))            #p=5: 자기회귀(AR) 항의 수       d=1: 차분의 수              q=0: 이동평균(MA) 항의 수
    model_fit= model.fit(disp=False)               #disp=False는 모델 훈련 과정의 출력을 표시하지 않도록 설정합니다.
    output=model_fit.predict()
    yhat=output.mean()                              #단일 예측값 선택: output[0]는 현재 시점의 예측값을 선택합니다.
    predictions.append(yhat)
    obs=test[t]                                      #관측값 저장: test[t]는 현재 시점의 실제 관측값입니다.
    history.append(obs)                               #히스토리에 추가: 실제 관측값을 history 리스트에 추가합니다. 이렇게 하면 이후 예측에 사용됩니다.
    print('predicted=%f, expected=%f' % (yhat, obs))
    
                 

C:\Users\user\AppData\Local\Temp\ipykernel_1036\3361153523.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('predicted=%f, expected=%f' % (yhat, obs))


predicted=202.919983, expected=346.300000
predicted=208.954809, expected=329.700000
predicted=212.667221, expected=445.400000
predicted=220.258910, expected=325.900000
predicted=224.260512, expected=449.300000
predicted=230.428286, expected=411.300000
predicted=236.292764, expected=417.400000
predicted=242.537781, expected=545.500000
predicted=249.036160, expected=477.600000
predicted=255.975667, expected=687.000000
predicted=268.827697, expected=435.300000
predicted=273.509390, expected=587.300000
predicted=282.293906, expected=676.900000


In [7]:
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [9]:
mse_value = mse(obs,  predictions)
mse_value 

192299.49269658665